In [80]:
# !pip install numpy-financial

## Imports

In [81]:
# Read stocks
import yfinance as yf

# To get analysts info
import yahoo_fin.stock_info as si

# Required to calculate net present value
import numpy_financial as npf

# For reading properties
from jproperties import Properties

## Load properties

In [82]:
configs = Properties()

with open('config/yf_dcf.properties', 'rb') as config_file:
     configs.load(config_file)

TICKER = configs.get('TICKER').data
# Perpetual growth rates is read as a string; we need to convert it to float
PERP_GROWTH_RATE = float(configs.get('PERP_GROWTH_RATE').data)
WACC_RATE = float(configs.get('WACC_RATE').data)

## Constants

In [83]:
PROJECTION_YEARS = 5

## Get Growth Rate

In [84]:
growth_est_df = si.get_analysts_info(TICKER)['Growth Estimates']
growth_str = growth_est_df[growth_est_df['Growth Estimates'] == 'Next 5 Years (per annum)'][TICKER].iloc[0]
growth_rate = round(float(growth_str.rstrip('%')) / 100.0, 4)
growth_rate

0.0636

## Calculate Change in WC

In [85]:
def calculate_cwc(balance_sheet):
    # Calculate the current wc
    payable_wc = balance_sheet.loc['Accounts Payable'][0] - balance_sheet.loc['Accounts Payable'][1]
    receivable_wc = balance_sheet.loc['Receivables'][0] - balance_sheet.loc['Receivables'][1]
    inventory_wc = balance_sheet.loc['Inventory'][0] - balance_sheet.loc['Inventory'][1]
    return payable_wc - receivable_wc - inventory_wc

## Calculate FCF

In [86]:
# Using yfinance to get other data
ticker = yf.Ticker(TICKER)

income_stmt_df = ticker.income_stmt
balance_sheet_df = ticker.balance_sheet
cash_flow_df = ticker.cash_flow

ebit = income_stmt_df.loc['EBIT'][0]
tax_rate = income_stmt_df.loc['Tax Rate For Calcs'][0]

# Change in working capital
cwc = calculate_cwc(balance_sheet_df)

depreciation_amoritization = cash_flow_df.loc['Depreciation And Amortization'][0]
cap_exp = cash_flow_df.loc['Capital Expenditure'][0]

# Calculate Unlevered Free Cash Flow 
ul_fcf = (ebit * (1-tax_rate)) + depreciation_amoritization + cwc + cap_exp

## Future Free Cash Flow

In [87]:
ffcf = []

# Year 1
ffcf.append(ul_fcf * (1 + growth_rate))

# Starting from 2nd year
for i in range(1, PROJECTION_YEARS):
    ffcf.append(ffcf[i-1] * (1 + growth_rate))

## Calculate the Terminal Value

In [88]:
# forecast_fcf[-1] refers to the last year in the growth period
terminal_value = ffcf[-1] * (1 + PERP_GROWTH_RATE)/(WACC_RATE - PERP_GROWTH_RATE)
# ffcf.append(terminal_value)

## Enterprise Valuation

In [89]:
# Add the terminal value to the last year
ffcf[-1] = ffcf[-1] + terminal_value

# Calculate dcf using npv - add zero or else the method assumes
# first value as the initial investment
enterprise_value = npf.npv(WACC_RATE, [0] + ffcf)
# enterprise_value = sum(pv_ffcf)

## Company Valuation

In [90]:
# Calculate Cash And Cash Equivalents
balance_sheet_df = ticker.balance_sheet
cash_and_equivalents = balance_sheet_df.loc['Cash And Cash Equivalents'][0]

# Current debt - only interested in the value
current_debt = balance_sheet_df.loc['Current Debt'][0]

# Long term debt - only interested in the value
long_term_debt = balance_sheet_df.loc['Long Term Debt'][0]

# Equity value -> enterprise_value + cash - total debt
equity_value = enterprise_value + cash_and_equivalents - (current_debt +  long_term_debt)

## Intrinsic Valuation

In [91]:
# Shares outstanding
shares_outstanding = ticker.info['sharesOutstanding']

# Intrinsic value
intrinsic_value = equity_value / shares_outstanding
'${:.2f}'.format(round(intrinsic_value,2)) + ' for ticker: ' + TICKER

'$107.41 for ticker: AAPL'

In [92]:
current_price = ticker.info['currentPrice']
upside = (intrinsic_value / current_price) - 1
'{:.2%}'.format(round(upside,4)) + ' for ticker: ' + TICKER

'-38.45% for ticker: AAPL'

In [93]:
if intrinsic_value > current_price:
    print('BUY')
else:
    print('SELL')

SELL


## Enterprise Valuation - alternative

In [94]:
# This is another way of calculating PV of FFCF - sum of it should match the npv calculation of npf
pv_ffcf = []
for idx, x in enumerate(ffcf):
    pv_ffcf.append(x/(1 + WACC_RATE)**(idx+1))
# This is equivalent to dcf
sum(pv_ffcf)

1785779807525.2454